In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
trans = transforms.Compose([
    transforms.ToTensor()
])
train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans)
data_loader = DataLoader(dataset=train_data, batch_size = 8, shuffle=True, num_workers=2)

In [4]:
#Define CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(6032, 120)
        self.fc2 = nn.Linear(120, 2)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [5]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.00001)

In [6]:
total_batch = len(data_loader)
training_epochs = 5
for epoch in range(training_epochs):
    avg_cost = 0
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/total_batch
    print("Epoch:{} cost:{}".format(epoch+1, avg_cost))

Epoch:1 cost:0.6831706166267395
Epoch:2 cost:0.6504665613174438
Epoch:3 cost:0.6104687452316284
Epoch:4 cost:0.5503584742546082
Epoch:5 cost:0.4711719751358032


In [11]:
#model 저장했다가 불러와서 쓰기
torch.save(model.state_dict(),'./model/model.pth')
new_model = CNN().to(device)
new_model.load_state_dict(torch.load('./model/model.pth'))

<All keys matched successfully>

In [13]:
#testing
trans = torchvision.transforms.Compose([
    transforms.Resize([64,128]),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = model(imgs)
        correct_prediction = torch.argmax(prediction,1) == label
        accuracy = correct_prediction.float().mean()
        print("Accuracy:", accuracy.item())

Accuracy: 1.0
